<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=9b1dfc9d1e9866e54245a072b801c2c6c4f34e8a00d9ad8f5f65d16f9f64c5fb
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.4
    Uninstalling protobuf-6.33.4:
      Successfully uninstalled protobuf-6.33.4


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-16 11:23:45
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.41 C
-------------------
Today PnL: 33.94 K (0.24%)
Current PnL: -28.52 L (-18.19%)
CY Booked + Current PnL: -14.23 L (-9.07%)
-------------------
Total profit:  1.92 L
Total loss:  -30.44 L
-------------------
Total Booked + Current PnL: 12.75 L (9.82%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (10.17%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 93.98 L (66.86%)
Deployed:  1.30 C
Current:  1.41 C
CAGR/XIRR %: 4.28%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,NESTLEIND,1377.00,0.65,15.98,4.67,21.40,14377.0,42424.0,307850.0,7.60,63.0,X-LC,6.60,14.0,2.95,2.19,23.32,XY25,NTT,FMCG
17,COALINDIA,484.83,-0.19,6.09,12.39,19.23,19777.0,9158.0,159618.0,24.27,69.0,L-LC,9.25,182.0,0.46,1.14,28.66,XY25,ATH,MINING
77,TTKPRESTIG,770.00,1.01,-22.27,28.76,0.09,22530.0,-22439.0,78338.0,81.86,37.0,M-SC,3.86,253.0,-1.00,0.56,2.92,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,0.82,-14.05,16.36,0.02,24547.0,-24519.0,150040.0,128.79,54.0,M-SC,9.09,240.0,-1.00,1.07,30.51,OX40N,NTT,PAINTS
49,MASFIN,398.61,0.87,-3.90,27.01,22.05,25431.0,-3825.0,94155.0,-17.34,47.0,H-SC,5.57,168.0,-0.15,0.67,36.86,XR,ATH,FINANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,7.86,-1.99,110.92,106.72,134082.0,-2454.0,120882.0,-51.39,58.0,H-SC,8.10,138.0,-0.02,0.86,20.92,AR,ATH,MISC
41,INFY,1972.00,5.23,14.60,17.14,34.24,61458.0,45676.0,358564.0,-10.62,64.0,X-LC,7.69,6.0,0.74,2.55,24.09,X40,NTT,IT
55,RAJESHEXPO,518.00,5.00,-66.52,198.87,0.07,91341.0,-91247.0,45930.0,1583.08,42.0,L-SC,0.75,269.0,-1.00,0.33,14.23,OX40N,NTT,JEWELLERY
13,BSOFT,831.70,4.44,-20.55,89.78,50.79,100333.0,-28899.0,111754.0,2.05,55.0,H-SC,9.95,171.0,-0.29,0.80,30.12,XR,ATH,IT
18,COFFEEDAY,80.00,4.38,-41.19,130.02,35.27,86825.0,-46771.0,66778.0,-55.73,48.0,L-SC,4.65,268.0,-0.54,0.48,63.44,XR,NTT,HOTELS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
71,TANLA,1943.92,-3.91,-47.50,318.50,119.69,452706.0,-128624.0,142137.0,-47.02,25.0,H-SC,12.59,150.0,-0.28,1.01,12.29,AR,ATH,IT
80,VALIANTORG,838.00,-2.86,-48.21,236.28,74.15,200195.0,-78877.0,84728.0,-252.21,34.0,H-SC,22.09,145.0,-0.39,0.60,8.61,XR,NTT,CHEMICALS
4,ALKYLAMINE,4546.37,-2.36,-31.69,183.46,93.63,126529.0,-31995.0,68968.0,-26.89,49.0,H-SC,16.54,143.0,-0.25,0.49,4.26,SR,ATH,CHEMICALS
47,KANSAINER,340.00,-2.17,-22.77,46.93,13.47,97741.0,-61398.0,208269.0,-68.31,46.0,H-SC,3.96,159.0,-0.63,1.48,5.90,XY24,NTT,PAINTS
46,JSWINFRA,342.00,-2.17,-12.32,32.79,16.43,63760.0,-27324.0,194450.0,-25.81,28.0,X-MC,7.45,66.0,-0.43,1.38,14.76,X40N,NTT,REALTY


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.21,-1.80,111.73,107.91,161987.0,-2663.0,144981.0,-20.15,53.0,M-SC,9.97,220.0,-0.02,1.03,8.10,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.82,-14.05,16.36,0.02,24547.0,-24519.0,150040.0,128.79,54.0,M-SC,9.09,240.0,-1.00,1.07,30.51,OX40N,NTT,PAINTS
72,TATAELXSI,9161.00,0.49,-20.27,65.60,32.03,68951.0,-26727.0,105108.0,-13.69,54.0,H-SC,6.59,157.0,-0.39,0.75,17.73,OX40N,NTT,IT
77,TTKPRESTIG,770.00,1.01,-22.27,28.76,0.09,22530.0,-22439.0,78338.0,81.86,37.0,M-SC,3.86,253.0,-1.00,0.56,2.92,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-2.17,-22.77,46.93,13.47,97741.0,-61398.0,208269.0,-68.31,46.0,H-SC,3.96,159.0,-0.63,1.48,5.90,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.00,1.12,1.26,19.24,20.74,39582.0,2558.0,205728.0,33.49,59.0,M-MC,5.17,189.0,0.06,1.46,12.01,XY25,NTT,FINANCE
17,COALINDIA,484.83,-0.19,6.09,12.39,19.23,19777.0,9158.0,159618.0,24.27,69.0,L-LC,9.25,182.0,0.46,1.14,28.66,XY25,ATH,MINING
84,WIPRO,420.00,2.73,9.63,57.13,72.27,106133.0,16326.0,185774.0,-6.08,56.0,M-LC,6.34,101.0,0.15,1.32,16.40,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,2.73,9.63,57.13,72.27,106133.0,16326.0,185774.0,-6.08,56.0,M-LC,6.34,101.0,0.15,1.32,16.40,XR,NTT,IT
38,INDIAMART,4810.62,7.86,-1.99,110.92,106.72,134082.0,-2454.0,120882.0,-51.39,58.0,H-SC,8.10,138.0,-0.02,0.86,20.92,AR,ATH,MISC
25,FINCABLES,1641.55,-0.21,-1.80,111.73,107.91,161987.0,-2663.0,144981.0,-20.15,53.0,M-SC,9.97,220.0,-0.02,1.03,8.10,OX40N,ATH,CABLES
37,IEX,219.00,-0.14,-5.55,57.36,48.63,108327.0,-11100.0,188854.0,-37.19,48.0,H-SC,13.23,137.0,-0.10,1.34,5.42,XR,NTT,MISC
49,MASFIN,398.61,0.87,-3.90,27.01,22.05,25431.0,-3825.0,94155.0,-17.34,47.0,H-SC,5.57,168.0,-0.15,0.67,36.86,XR,ATH,FINANCE


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-0.99,-18.76,36.37,10.78,71003.0,-45094.0,195224.0,-51.78,12.0,X-LC,9.32,1.0,-0.64,1.39,0.00,X40,NTT,FMCG
7,AWL,485.00,-1.70,-30.57,126.41,57.19,286218.0,-99707.0,226420.0,-67.20,18.0,X-MC,11.00,58.0,-0.35,1.61,0.00,XY24,NTT,FMCG
22,DIXON,18931.72,-1.22,-23.96,72.62,31.26,127428.0,-55296.0,175472.0,-64.52,24.0,X-MC,21.49,56.0,-0.43,1.25,0.00,X40N,ATH,IT
11,BATAINDIA,2096.00,0.10,-41.52,131.18,35.20,98716.0,-53418.0,75252.0,-1.37,27.0,X-SC,17.59,93.0,-0.54,0.54,0.10,X40,NTT,FOOTWEAR
46,JSWINFRA,342.00,-2.17,-12.32,32.79,16.43,63760.0,-27324.0,194450.0,-25.81,28.0,X-MC,7.45,66.0,-0.43,1.38,14.76,X40N,NTT,REALTY


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
45,JIOFIN,387.00,-1.20,-8.79,36.48,24.48,88958.0,-23512.0,243853.0,-10.35,36.0,X-LC,1.67,37.0,-0.26,1.74,41.34,XY24,BTT,FINANCE
73,TCS,4389.97,0.23,-12.01,37.19,20.71,123768.0,-45437.0,332800.0,-24.27,47.0,X-LC,1.88,3.0,-0.37,2.37,11.20,X40,ATH,IT
78,UNITDSPR,1644.00,1.14,-0.74,21.67,20.77,50362.0,-1738.0,232406.0,-9.42,40.0,X-MC,1.96,70.0,-0.03,1.65,6.12,X40N,NTT,BREWERIES
20,DABUR,735.00,0.98,1.43,41.67,43.70,104849.0,3550.0,251618.0,-6.86,59.0,X-MC,2.15,73.0,0.03,1.79,16.70,XY24,BTT,FMCG
65,SIEMENS,4671.50,-0.66,-20.71,58.29,25.51,86013.0,-38535.0,147560.0,-9.42,36.0,H-LC,2.47,50.0,-0.45,1.05,8.74,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,AWL,485.00,-1.70,-30.57,126.41,57.19,286218.0,-99707.0,226420.0,-67.20,18.0,X-MC,11.00,58.0,-0.35,1.61,0.0,XY24,NTT,FMCG
8,BAJAJHFL,181.50,-0.58,-20.38,95.27,55.47,184546.0,-49578.0,193708.0,-29.54,33.0,X-MC,15.89,63.0,-0.27,1.38,0.0,X40N,ATH,FINANCE
9,BAJAJHLDNG,14451.52,-0.45,-4.39,35.15,29.22,63896.0,-8344.0,181781.0,-6.67,35.0,X-LC,11.98,29.0,-0.13,1.29,0.0,X40,ATH,FINANCE
22,DIXON,18931.72,-1.22,-23.96,72.62,31.26,127428.0,-55296.0,175472.0,-64.52,24.0,X-MC,21.49,56.0,-0.43,1.25,0.0,X40N,ATH,IT
58,RELAXO,1176.00,-0.56,-49.37,200.84,52.31,147595.0,-71671.0,73489.0,-46.08,33.0,X-SC,8.03,92.0,-0.49,0.52,0.0,X40N,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,-1.65,-29.96,103.94,42.84,47324.0,-19476.0,45530.0,-54.27,43.0,X-SC,24.88,83.0,-0.41,0.32,3.01,XY24,NTT,MISC
58,RELAXO,1176.00,-0.56,-49.37,200.84,52.31,147595.0,-71671.0,73489.0,-46.08,33.0,X-SC,8.03,92.0,-0.49,0.52,0.00,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.10,-41.52,131.18,35.20,98716.0,-53418.0,75252.0,-1.37,27.0,X-SC,17.59,93.0,-0.54,0.54,0.10,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,-0.61,-3.52,28.60,24.08,35030.0,-4467.0,122483.0,-11.26,43.0,X-SC,5.85,91.0,-0.13,0.87,15.41,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.15,-17.73,74.20,43.32,99428.0,-28872.0,134000.0,-29.29,45.0,X-SC,8.19,90.0,-0.29,0.95,2.89,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-0.99,-18.76,36.37,10.78,71003.0,-45094.0,195224.0,-51.78,12.0,X-LC,9.32,1.0,-0.64,1.39,0.00,X40,NTT,FMCG
75,TMPV,600.00,2.06,-26.83,68.07,22.98,112514.0,-60607.0,165291.0,-24.30,47.0,X-LC,3.61,2.0,-0.54,1.18,3.96,XY24,NTT,AUTO
73,TCS,4389.97,0.23,-12.01,37.19,20.71,123768.0,-45437.0,332800.0,-24.27,47.0,X-LC,1.88,3.0,-0.37,2.37,11.20,X40,ATH,IT
81,VBL,671.64,-0.19,1.29,33.94,35.67,108583.0,4083.0,319925.0,-10.77,60.0,X-LC,2.78,5.0,0.04,2.28,15.33,X40N,ATH,FMCG
41,INFY,1972.00,5.23,14.60,17.14,34.24,61458.0,45676.0,358564.0,-10.62,64.0,X-LC,7.69,6.0,0.74,2.55,24.09,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.63,-29.83,89.37,32.89,48419.0,-23028.0,54178.0,-769.18,73.0,L-MC,9.19,259.0,-0.48,0.39,49.20,XR,NTT,BANKS
13,BSOFT,831.70,4.44,-20.55,89.78,50.79,100333.0,-28899.0,111754.0,2.05,55.0,H-SC,9.95,171.0,-0.29,0.80,30.12,XR,ATH,IT
69,SURYODAY,216.00,2.74,-17.78,49.64,23.03,73088.0,-31834.0,147237.0,60.09,56.0,H-SC,4.54,165.0,-0.44,1.05,46.09,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,0.82,-14.05,16.36,0.02,24547.0,-24519.0,150040.0,128.79,54.0,M-SC,9.09,240.0,-1.00,1.07,30.51,OX40N,NTT,PAINTS
17,COALINDIA,484.83,-0.19,6.09,12.39,19.23,19777.0,9158.0,159618.0,24.27,69.0,L-LC,9.25,182.0,0.46,1.14,28.66,XY25,ATH,MINING


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,-1.27,-28.10,57.02,12.90,89580.0,-61394.0,157102.0,131.97,60.0,H-SC,8.97,173.0,-0.69,1.12,30.63,OX40N,NTT,FINANCE
17,COALINDIA,484.83,-0.19,6.09,12.39,19.23,19777.0,9158.0,159618.0,24.27,69.0,L-LC,9.25,182.0,0.46,1.14,28.66,XY25,ATH,MINING
40,INDUSINDBK,1800.00,0.63,-29.83,89.37,32.89,48419.0,-23028.0,54178.0,-769.18,73.0,L-MC,9.19,259.0,-0.48,0.39,49.20,XR,NTT,BANKS
57,RECLTD,446.00,1.12,1.26,19.24,20.74,39582.0,2558.0,205728.0,33.49,59.0,M-MC,5.17,189.0,0.06,1.46,12.01,XY25,NTT,FINANCE
72,TATAELXSI,9161.00,0.49,-20.27,65.60,32.03,68951.0,-26727.0,105108.0,-13.69,54.0,H-SC,6.59,157.0,-0.39,0.75,17.73,OX40N,NTT,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.51
1,25,44.55
2,50,76.53


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.71
MC    30.50
LC    26.79
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.30
X40      24.04
X40N     12.73
XY25     10.83
XR        9.02
AR        8.42
OX40N     7.71
SR        0.95
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.12
X-LC    23.28
H-SC    22.86
M-SC    11.20
X-SC     7.84
H-MC     4.53
M-MC     1.46
M-LC     1.32
L-LC     1.14
H-LC     1.05
L-SC     0.81
L-MC     0.39
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.16,-9.25,45.09
IT,12.88,-18.08,77.64
FINANCE,11.11,-14.21,62.15
MISC,7.08,-30.43,83.79
ELECTRICAL,5.83,-14.47,56.40
PAINTS,5.81,-13.00,29.79
INSURANCE,4.70,-2.70,37.84
PHARMA,3.93,-6.38,39.87
BANKS,2.82,-35.27,117.24


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3276230.0,21
AR,1386942.0,10
XR,1226082.0,12
X40,1141474.0,15
X40N,952290.0,9
OX40N,708785.0,10
XY25,419423.0,7
SR,286376.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3581833.0,24
X-MC,1641757.0,16
M-SC,1485222.0,15
X-LC,952736.0,13
X-SC,816371.0,8
H-MC,441593.0,3
L-SC,178166.0,2
M-LC,106133.0,1
H-LC,86013.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1258971.0      6
           AR         935537.0      5
M-SC       XY24       851906.0      6
H-SC       XR         794246.0      7
X-MC       X40        532305.0      7
           XY24       455642.0      3
           X40N       426096.0      4
X-LC       X40        413950.0      6
X-SC       X40N       321230.0      3
M-SC       OX40N      310741.0      5
H-SC       OX40N      306703.0      4
X-SC       XY24       299922.0      3
H-SC       SR         286376.0      2
H-MC       AR         233276.0      2
X-MC       XY25       227714.0      2
H-MC       XY24       208317.0      1
X-LC       X40N       204964.0      2
           XY24       201472.0      2
X-SC       X40        195219.0      2
M-SC       XR         190459.0      2
X-LC       XY25       132350.0      3
M-SC       AR         132116.0      2
M-LC       XR         106133.0      1
L-SC       OX40N       91341.0      1
           XR          86825.0      1
H-LC       AR          86013.0      1
L-MC       XR          48419.0      1
M-MC       XY25        39582.0      1
L-LC       XY25        19777.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 31.0 seconds
